In [1]:
# Dependencies

import pandas as pd
import numpy as np
import glob
import os
import timeit                                # To check performance
from datetime import datetime, date, time, timedelta
import matplotlib.pyplot as plt

In [2]:
# Import CSV

tic = timeit.default_timer()                            # Monitor performance

ecobici_rides_df = pd.read_csv("dataset_export_files/ecobici_ride_data-oneInHundredv2.csv", index_col=None, header=0)

toc = timeit.default_timer()                            # Monitor performance
print(f'Time (in seconds) to import CSV file: {round(toc - tic, 2)}')

Time (in seconds) to import CSV file: 0.57


In [3]:
ecobici_rides_df_test = ecobici_rides_df.copy()
ecobici_rides_df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707039 entries, 0 to 707038
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Genero_Usuario         707039 non-null  object 
 1   Edad_Usuario           707039 non-null  int64  
 2   Ciclo_Estacion_Retiro  707039 non-null  int64  
 3   Ciclo_Estacion_Arribo  707039 non-null  int64  
 4   Usage_Timestamp        707039 non-null  object 
 5   Duration(Min)          707039 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 32.4+ MB


In [4]:
ecobici_rides_df_test['Usage_Timestamp'] = pd.to_datetime(ecobici_rides_df_test['Usage_Timestamp'])


,Genero_Usuario,Edad_Usuario,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Usage_Timestamp,Duration(Min)
0,M,28,85,85,2010-02-16 12:45:37.427,3.1
1,M,35,78,63,2010-02-19 13:52:30.700,12.1
2,M,34,85,23,2010-02-21 14:30:46.390,26.8
3,M,46,20,17,2010-02-22 16:17:10.470,6.4
4,F,28,74,46,2010-02-23 18:51:57.057,14.9
...,...,...,...,...,...,...
707034,M,27,316,54,2021-12-05 09:29:40.000,13.3
707035,M,49,158,83,2021-12-05 09:24:30.000,10.8
707036,M,33,417,313,2021-12-05 08:55:46.000,12.1
707037,F,23,467,260,2021-12-05 08:54:12.000,33.7


## Routes

In [5]:
ecobici_station_groupby = ecobici_rides_df_test.groupby(ecobici_rides_df_test.Usage_Timestamp.dt.year).agg({"Ciclo_Estacion_Retiro": "unique"})

ecobici_station_groupby

# .agg({"Usage_Timestamp": "count"}).unstack()


,Ciclo_Estacion_Retiro
Usage_Timestamp,
2010,"[85, 78, 20, 74, 27, 23, 4, 15, 19, 76, 6, 34,..."
2011,"[76, 21, 43, 59, 5, 61, 37, 24, 64, 36, 27, 6,..."
2012,"[60, 70, 37, 27, 85, 44, 41, 90, 30, 61, 68, 3..."
2013,"[78, 155, 117, 211, 212, 57, 72, 127, 180, 73,..."
2014,"[82, 270, 101, 156, 175, 269, 23, 71, 24, 27, ..."
2015,"[195, 42, 66, 147, 188, 10, 56, 120, 19, 27, 9..."
2016,"[32, 31, 70, 412, 321, 82, 160, 53, 125, 91, 1..."
2017,"[59, 211, 17, 107, 135, 224, 163, 6, 156, 233,..."
2018,"[405, 452, 384, 173, 34, 149, 288, 233, 61, 15..."


In [6]:
for index, row in ecobici_station_groupby.iterrows():
    ecobici_station_groupby['Station_Count'] = len(row['Ciclo_Estacion_Retiro'])

ecobici_station_groupby

,Ciclo_Estacion_Retiro,Station_Count
Usage_Timestamp,,
2010,"[85, 78, 20, 74, 27, 23, 4, 15, 19, 76, 6, 34,...",479
2011,"[76, 21, 43, 59, 5, 61, 37, 24, 64, 36, 27, 6,...",479
2012,"[60, 70, 37, 27, 85, 44, 41, 90, 30, 61, 68, 3...",479
2013,"[78, 155, 117, 211, 212, 57, 72, 127, 180, 73,...",479
2014,"[82, 270, 101, 156, 175, 269, 23, 71, 24, 27, ...",479
2015,"[195, 42, 66, 147, 188, 10, 56, 120, 19, 27, 9...",479
2016,"[32, 31, 70, 412, 321, 82, 160, 53, 125, 91, 1...",479
2017,"[59, 211, 17, 107, 135, 224, 163, 6, 156, 233,...",479
2018,"[405, 452, 384, 173, 34, 149, 288, 233, 61, 15...",479


In [17]:
ecobici_station_groupby.loc[2017, 'Ciclo_Estacion_Retiro']


array([  59,  211,   17,  107,  135,  224,  163,    6,  156,  233,  200,
        306,  133,   15,   52,   83,  115,  295,  139,   31,  342,  143,
        154,   21,   23,  194,  413,  271,  270,  111,  130,  288,  278,
         16,  252,   50,  195,   18,   89,  165,   68,  242,  113,   28,
        215,   27,  150,   34,  321,  329,  296,  134,   57,  433,   76,
        440,  182,    1,  448,  183,  286,  388,    9,   10,  235,   46,
         80,  324,   62,  208,   56,  274,  219,  232,  305,  172,  159,
        141,  198,   63,   32,  217,  291,  374,   20,  103,   11,  216,
         40,   51,  213,   84,  417,  112,  371,  317,  261,  292,   22,
        249,  302,  356,  449,  227,  358,  157,  390,  127,  425,   85,
        238,  260,  212,  153,  373,   95,  359,   97,  142,  190,  206,
        136,  181,   55,   74,   98,  161,  379,   58,  355,  229,  330,
        389,  381,  210,  447,  117,  114,  326,  299,  205,   64,  387,
        346,   39,   87,    3,  426,  110,  151,   

In [ ]:
## ecobici_station_groupby.loc[2021, 'Ciclo_Estacion_Retiro']

In [20]:
routes_2010 = ecobici_rides_df_test.loc[ecobici_rides_df_test.Usage_Timestamp.dt.year==2010]
routes_2010

,Genero_Usuario,Edad_Usuario,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Usage_Timestamp,Duration(Min)
0,M,28,85,85,2010-02-16 12:45:37.427,3.1
1,M,35,78,63,2010-02-19 13:52:30.700,12.1
2,M,34,85,23,2010-02-21 14:30:46.390,26.8
3,M,46,20,17,2010-02-22 16:17:10.470,6.4
4,F,28,74,46,2010-02-23 18:51:57.057,14.9
...,...,...,...,...,...,...
8406,M,38,75,37,2010-12-31 08:21:22.327,7.0
8407,F,36,51,28,2010-12-31 09:39:04.680,9.6
8408,M,34,32,4,2010-12-31 09:07:39.400,6.8
8409,M,31,78,46,2010-12-31 12:46:17.450,6.9


In [23]:
unique_routes_2010 = uniqueRoutes(routes_2010)
sorted_unique_routes_2010 = dict(sorted(unique_routes_2010.items(), key=lambda item: item[1], reverse=True))
print(f'Total Count of Routes: {len(sorted_unique_routes_2010)}')
sorted_unique_routes_2010

Total Count of Routes: 3722


{(61, 61): 25,
 (85, 85): 24,
 (70, 70): 22,
 (13, 20): 18,
 (20, 17): 17,
 (20, 13): 17,
 (19, 85): 16,
 (18, 1): 15,
 (74, 74): 14,
 (17, 20): 14,
 (67, 67): 14,
 (36, 53): 14,
 (25, 27): 13,
 (1, 27): 13,
 (31, 28): 13,
 (58, 58): 13,
 (27, 27): 13,
 (63, 63): 13,
 (23, 27): 12,
 (63, 64): 12,
 (18, 15): 12,
 (84, 76): 12,
 (63, 84): 12,
 (64, 36): 12,
 (16, 13): 12,
 (67, 77): 12,
 (49, 49): 11,
 (76, 64): 11,
 (2, 9): 11,
 (32, 24): 11,
 (72, 72): 11,
 (13, 13): 11,
 (84, 58): 11,
 (64, 58): 10,
 (61, 64): 10,
 (70, 61): 10,
 (84, 63): 10,
 (53, 36): 10,
 (73, 73): 10,
 (18, 18): 10,
 (54, 54): 10,
 (1, 6): 10,
 (37, 64): 10,
 (4, 4): 9,
 (80, 80): 9,
 (61, 84): 9,
 (19, 19): 9,
 (9, 1): 9,
 (60, 72): 9,
 (19, 17): 9,
 (20, 20): 9,
 (36, 57): 9,
 (17, 27): 9,
 (35, 17): 9,
 (79, 79): 9,
 (1, 34): 9,
 (1, 9): 9,
 (71, 76): 9,
 (53, 62): 9,
 (27, 23): 9,
 (34, 1): 9,
 (18, 13): 9,
 (58, 84): 9,
 (1, 4): 9,
 (84, 61): 9,
 (76, 76): 9,
 (49, 46): 9,
 (53, 66): 8,
 (24, 27): 8,
 (36, 6

In [26]:
lst = []

for (key, value) in sorted_unique_routes_2010.items():
   # Check if key is even then add pair to new dictionary
   lst.append([value, key[0], key[1]])

sorted_unique_routes_2010_df = pd.DataFrame(lst, columns=['Total_Rides',
                                                       'Start_Station',
                                                       'End_Station'])

sorted_unique_routes_2010_df

,Total_Rides,Start_Station,End_Station
0,25,61,61
1,24,85,85
2,22,70,70
3,18,13,20
4,17,20,17
...,...,...,...
3717,1,28,41
3718,1,53,49
3719,1,46,51
3720,1,51,28


In [17]:
def uniqueRoutes(yearRides):

    routes = {}

    for index, row in yearRides.iterrows():
        try:
            route = (row['Ciclo_Estacion_Retiro'], row['Ciclo_Estacion_Arribo'])
            if route not in routes: 
                routes[route] = 1
            else:
                routes[route] +=1
        except: pass

    return routes

In [18]:
ecobici_routes_groupby = ecobici_rides_df_test.groupby(ecobici_rides_df_test.Usage_Timestamp.dt.year).agg(uniqueRoutes(ecobici_rides_df_test))

ecobici_routes_groupby

SpecificationError: Column(s) [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (1, 33), (1, 34), (1, 35), (1, 36), (1, 37), (1, 38), (1, 39), (1, 40), (1, 41), (1, 42), (1, 43), (1, 44), (1, 45), (1, 46), (1, 47), (1, 48), (1, 49), (1, 50), (1, 51), (1, 52), (1, 53), (1, 54), (1, 55), (1, 56), (1, 57), (1, 58), (1, 59), (1, 60), (1, 61), (1, 62), (1, 63), (1, 64), (1, 65), (1, 66), (1, 67), (1, 68), (1, 69), (1, 70), (1, 71), (1, 72), (1, 73), (1, 74), (1, 75), (1, 76), (1, 77), (1, 78), (1, 79), (1, 80), (1, 81), (1, 82), (1, 83), (1, 84), (1, 85), (1, 86), (1, 87), (1, 88), (1, 89), (1, 90), (1, 91), (1, 92), (1, 93), (1, 94), (1, 95), (1, 96), (1, 97), (1, 98), (1, 99), (1, 100), (1, 101), (1, 102), (1, 103), (1, 104), (1, 105), (1, 106), (1, 107), (1, 108

In [89]:
top_routes = {}
# Iterate over all the items in dictionary and filter items which has even keys
for (key, value) in sorted_routes.items():
   # Check if key is even then add pair to new dictionary
   if value > 70:
        top_routes[key] = value

print(f'Total Count of Routes: {len(top_routes)}')
top_routes

Total Count of Routes: 945


{(18, 1): 477,
 (211, 217): 434,
 (1, 18): 427,
 (174, 183): 342,
 (41, 3): 335,
 (23, 27): 328,
 (183, 174): 318,
 (61, 61): 309,
 (217, 211): 305,
 (34, 1): 298,
 (36, 53): 292,
 (1, 35): 291,
 (21, 27): 290,
 (70, 70): 281,
 (27, 43): 281,
 (27, 29): 278,
 (35, 1): 274,
 (27, 1): 268,
 (41, 19): 265,
 (1, 27): 259,
 (85, 174): 257,
 (53, 36): 250,
 (27, 27): 250,
 (208, 206): 248,
 (165, 163): 242,
 (24, 27): 238,
 (206, 208): 234,
 (208, 242): 231,
 (151, 158): 228,
 (84, 71): 225,
 (84, 63): 224,
 (270, 266): 224,
 (27, 41): 221,
 (174, 85): 218,
 (208, 197): 217,
 (84, 61): 215,
 (25, 27): 214,
 (41, 10): 214,
 (77, 68): 213,
 (32, 4): 213,
 (1, 1): 212,
 (15, 27): 211,
 (27, 15): 210,
 (135, 136): 208,
 (64, 45): 207,
 (84, 64): 205,
 (73, 77): 205,
 (1, 86): 205,
 (28, 27): 204,
 (84, 59): 204,
 (41, 20): 201,
 (41, 18): 199,
 (197, 208): 198,
 (256, 183): 197,
 (84, 74): 195,
 (158, 141): 194,
 (77, 73): 191,
 (77, 66): 191,
 (57, 36): 190,
 (7, 7): 188,
 (22, 27): 187,
 (174,

In [96]:
top_routes_list = []

for (key, value) in top_routes.items():
   # Check if key is even then add pair to new dictionary
   top_routes_list.append([value, key[0], key[1]])

top_routes_list


[[477, 18, 1],
 [434, 211, 217],
 [427, 1, 18],
 [342, 174, 183],
 [335, 41, 3],
 [328, 23, 27],
 [318, 183, 174],
 [309, 61, 61],
 [305, 217, 211],
 [298, 34, 1],
 [292, 36, 53],
 [291, 1, 35],
 [290, 21, 27],
 [281, 70, 70],
 [281, 27, 43],
 [278, 27, 29],
 [274, 35, 1],
 [268, 27, 1],
 [265, 41, 19],
 [259, 1, 27],
 [257, 85, 174],
 [250, 53, 36],
 [250, 27, 27],
 [248, 208, 206],
 [242, 165, 163],
 [238, 24, 27],
 [234, 206, 208],
 [231, 208, 242],
 [228, 151, 158],
 [225, 84, 71],
 [224, 84, 63],
 [224, 270, 266],
 [221, 27, 41],
 [218, 174, 85],
 [217, 208, 197],
 [215, 84, 61],
 [214, 25, 27],
 [214, 41, 10],
 [213, 77, 68],
 [213, 32, 4],
 [212, 1, 1],
 [211, 15, 27],
 [210, 27, 15],
 [208, 135, 136],
 [207, 64, 45],
 [205, 84, 64],
 [205, 73, 77],
 [205, 1, 86],
 [204, 28, 27],
 [204, 84, 59],
 [201, 41, 20],
 [199, 41, 18],
 [198, 197, 208],
 [197, 256, 183],
 [195, 84, 74],
 [194, 158, 141],
 [191, 77, 73],
 [191, 77, 66],
 [190, 57, 36],
 [188, 7, 7],
 [187, 22, 27],
 [187,

In [99]:

top_routes_df = pd.DataFrame(top_routes_list, columns=['Total_Rides',
                                                       'Start_Station',
                                                       'End_Station'])

top_routes_df

,Total_Rides,Start_Station,End_Station
0,477,18,1
1,434,211,217
2,427,1,18
3,342,174,183
4,335,41,3
...,...,...,...
940,71,182,180
941,71,267,18
942,71,271,23
943,71,272,18


In [33]:
stations = pd.read_csv("../Estaciones.csv", index_col=None, header=0)
stations.head()

,ID,Name,Lat,Lon,districtName
0,124,124 CLAUDIO BERNARD-DR. LICEAGA,19.422392,-99.150358,Ampliacion Granada
1,159,159 HUATABAMPO-EJE 1 PTE. AV. CUAUHTÉMOC,19.407517,-99.155373,Ampliacion Granada
2,241,E241 EJERCITO NAL-JUAN VAZQUEZ DE LA MELLA,19.438620,-99.207580,Ampliacion Granada
3,243,243 MIGUEL DE CERVANTES SAAVEDRA-LAGO FILT,19.440839,-99.196712,Ampliacion Granada
4,350,350 JOSE CLEMENTE OROZCO-CORREGGIO,19.384062,-99.181482,Ampliacion Granada


In [34]:
stations_test = stations.copy()
stations_test

,ID,Name,Lat,Lon,districtName
0,124,124 CLAUDIO BERNARD-DR. LICEAGA,19.422392,-99.150358,Ampliacion Granada
1,159,159 HUATABAMPO-EJE 1 PTE. AV. CUAUHTÉMOC,19.407517,-99.155373,Ampliacion Granada
2,241,E241 EJERCITO NAL-JUAN VAZQUEZ DE LA MELLA,19.438620,-99.207580,Ampliacion Granada
3,243,243 MIGUEL DE CERVANTES SAAVEDRA-LAGO FILT,19.440839,-99.196712,Ampliacion Granada
4,350,350 JOSE CLEMENTE OROZCO-CORREGGIO,19.384062,-99.181482,Ampliacion Granada
...,...,...,...,...,...
475,380,380 MORAS-SAN LORENZO,19.374876,-99.173512,Tlacoquemecatl
476,440,440 REAL DE MAYORAZGO-PUENTE XOCO,19.360650,-99.168669,Xoco
477,441,441 PUENTE XOCO-REAL DE MAYORAZGO,19.361215,-99.167527,Xoco
478,442,442 REAL DE MAYORAZGO-AVENIDA MÉXICO-COYOACAN,19.359905,-99.165300,Xoco


In [35]:
stations_sorted = stations_test.sort_values(by=['ID'])
stations_sorted.head(20)

,ID,Name,Lat,Lon,districtName
111,1,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,Cuauhtemoc
112,2,2 RIO GUADALQUIVIR-RIO BALSAS,19.431386,-99.171695,Cuauhtemoc
259,3,3 REFORMA-INSURGENTES,19.431655,-99.158668,Juarez
107,4,4 RIO NILO-RIO PANUCO,19.428309,-99.171713,Cuauhtemoc
108,5,5 RIO PANUCO-RIO TIBER,19.429720,-99.169363,Cuauhtemoc
109,6,6 RIO PANUCO-RIO SENA,19.430922,-99.166959,Cuauhtemoc
110,7,7 RIO ELBA-RIO LERMA,19.425468,-99.175166,Cuauhtemoc
260,8,8 REFORMA-GENERAL PRIM,19.431141,-99.158317,Juarez
113,9,9 RIO GUADALQUIVIR-RIO LERMA,19.428009,-99.169918,Cuauhtemoc
466,10,10 REFORMA-RAMIREZ,19.433321,-99.154752,Tabacalera


In [36]:
sorted_unique_routes_2010_df = pd.merge(sorted_unique_routes_2010_df, stations_sorted, left_on=["Start_Station"],
                          right_on=["ID"],
                          how = 'left')
            
sorted_unique_routes_2010_df


,Total_Rides,Start_Station,End_Station,ID,Name,Lat,Lon,districtName
0,25,61,61,61,61 AVENIDA MEXICO-SONORA,19.413742,-99.169525,Hipodromo
1,24,85,85,85,85 ROSAS MORENO-SULLIVAN,19.434250,-99.162508,San Rafael
2,22,70,70,70,70 PARQUE MEXICO-MICHOACAN,19.411272,-99.169718,Hipodromo
3,18,13,20,13,13 REFORMA-RIO MISSISSIPPI,19.425020,-99.172310,Cuauhtemoc
4,17,20,17,20,20 REFORMA-RIO TAMESIS,19.430964,-99.160206,Cuauhtemoc
...,...,...,...,...,...,...,...,...
3717,1,28,41,28,28 TOLEDO-TOKIO,19.423542,-99.172494,Juarez
3718,1,53,49,53,53 FERNANDO MONTES DE OCA-TULA,19.414773,-99.178433,Condesa
3719,1,46,51,46,46 DURANGO-SALAMANCA,19.419214,-99.169732,Roma Norte
3720,1,51,28,51,51 COLIMA-OAXACA,19.417995,-99.168248,Roma Norte


In [37]:
sorted_unique_routes_2010_df = pd.merge(sorted_unique_routes_2010_df, stations_sorted, left_on=["End_Station"],
                          right_on=["ID"],
                          how = 'left')

sorted_unique_routes_2010_df.head()


,Total_Rides,Start_Station,End_Station,ID_x,Name_x,Lat_x,Lon_x,districtName_x,ID_y,Name_y,Lat_y,Lon_y,districtName_y
0,25,61,61,61,61 AVENIDA MEXICO-SONORA,19.413742,-99.169525,Hipodromo,61,61 AVENIDA MEXICO-SONORA,19.413742,-99.169525,Hipodromo
1,24,85,85,85,85 ROSAS MORENO-SULLIVAN,19.434250,-99.162508,San Rafael,85,85 ROSAS MORENO-SULLIVAN,19.434250,-99.162508,San Rafael
2,22,70,70,70,70 PARQUE MEXICO-MICHOACAN,19.411272,-99.169718,Hipodromo,70,70 PARQUE MEXICO-MICHOACAN,19.411272,-99.169718,Hipodromo
3,18,13,20,13,13 REFORMA-RIO MISSISSIPPI,19.425020,-99.172310,Cuauhtemoc,20,20 REFORMA-RIO TAMESIS,19.430964,-99.160206,Cuauhtemoc
4,17,20,17,20,20 REFORMA-RIO TAMESIS,19.430964,-99.160206,Cuauhtemoc,17,17 REFORMA-RIO TIBER,19.427501,-99.167125,Cuauhtemoc


In [39]:
sorted_unique_routes_2010_df.drop(['ID_x', 'ID_y'], inplace=True, axis=1)
sorted_unique_routes_2010_df

,Total_Rides,Start_Station,End_Station,Name_x,Lat_x,Lon_x,districtName_x,Name_y,Lat_y,Lon_y,districtName_y
0,25,61,61,61 AVENIDA MEXICO-SONORA,19.413742,-99.169525,Hipodromo,61 AVENIDA MEXICO-SONORA,19.413742,-99.169525,Hipodromo
1,24,85,85,85 ROSAS MORENO-SULLIVAN,19.434250,-99.162508,San Rafael,85 ROSAS MORENO-SULLIVAN,19.434250,-99.162508,San Rafael
2,22,70,70,70 PARQUE MEXICO-MICHOACAN,19.411272,-99.169718,Hipodromo,70 PARQUE MEXICO-MICHOACAN,19.411272,-99.169718,Hipodromo
3,18,13,20,13 REFORMA-RIO MISSISSIPPI,19.425020,-99.172310,Cuauhtemoc,20 REFORMA-RIO TAMESIS,19.430964,-99.160206,Cuauhtemoc
4,17,20,17,20 REFORMA-RIO TAMESIS,19.430964,-99.160206,Cuauhtemoc,17 REFORMA-RIO TIBER,19.427501,-99.167125,Cuauhtemoc
...,...,...,...,...,...,...,...,...,...,...,...
3717,1,28,41,28 TOLEDO-TOKIO,19.423542,-99.172494,Juarez,41 REFORMA-AV. DE LA REPUBLICA,19.435637,-99.150137,Tabacalera
3718,1,53,49,53 FERNANDO MONTES DE OCA-TULA,19.414773,-99.178433,Condesa,49 MAZATLAN-JUAN DE LA BARRERA,19.416712,-99.175704,Condesa
3719,1,46,51,46 DURANGO-SALAMANCA,19.419214,-99.169732,Roma Norte,51 COLIMA-OAXACA,19.417995,-99.168248,Roma Norte
3720,1,51,28,51 COLIMA-OAXACA,19.417995,-99.168248,Roma Norte,28 TOLEDO-TOKIO,19.423542,-99.172494,Juarez


In [40]:
sorted_unique_routes_2010_df = sorted_unique_routes_2010_df.rename(columns={"Name_x": "Start_Station_Name",
                                               'Lat_x' : 'Start_Station_Lat',
                                               'Lon_x' : 'Start_Station_Lon',
                                               'districtName_x' : 'Start_Colonia',
                                               "Name_y": "End_Station_Name",
                                               'Lat_y' : 'End_Station_Lat',
                                               'Lon_y' : 'End_Station_Lon',
                                               'districtName_y' : 'End_Colonia'})
sorted_unique_routes_2010_df.head()

,Total_Rides,Start_Station,End_Station,Start_Station_Name,Start_Station_Lat,Start_Station_Lon,Start_Colonia,End_Station_Name,End_Station_Lat,End_Station_Lon,End_Colonia
0,25,61,61,61 AVENIDA MEXICO-SONORA,19.413742,-99.169525,Hipodromo,61 AVENIDA MEXICO-SONORA,19.413742,-99.169525,Hipodromo
1,24,85,85,85 ROSAS MORENO-SULLIVAN,19.434250,-99.162508,San Rafael,85 ROSAS MORENO-SULLIVAN,19.434250,-99.162508,San Rafael
2,22,70,70,70 PARQUE MEXICO-MICHOACAN,19.411272,-99.169718,Hipodromo,70 PARQUE MEXICO-MICHOACAN,19.411272,-99.169718,Hipodromo
3,18,13,20,13 REFORMA-RIO MISSISSIPPI,19.425020,-99.172310,Cuauhtemoc,20 REFORMA-RIO TAMESIS,19.430964,-99.160206,Cuauhtemoc
4,17,20,17,20 REFORMA-RIO TAMESIS,19.430964,-99.160206,Cuauhtemoc,17 REFORMA-RIO TIBER,19.427501,-99.167125,Cuauhtemoc


In [41]:
sorted_unique_routes_2010_df.to_json("Routes/sorted_unique_routes_2010.json", orient='index') 

In [125]:
top_routes_df3.to_csv("routes_to_plot.csv", index=False) 

In [42]:
routes_2018 = ecobici_rides_df_test.loc[ecobici_rides_df_test.Usage_Timestamp.dt.year==2018]

unique_routes_2018 = uniqueRoutes(routes_2018)
sorted_unique_routes_2018 = dict(sorted(unique_routes_2018.items(), key=lambda item: item[1], reverse=True))

lst = []

for (key, value) in sorted_unique_routes_2018.items():
   # Check if key is even then add pair to new dictionary
   lst.append([value, key[0], key[1]])

sorted_unique_routes_2018_df = pd.DataFrame(lst, columns=['Total_Rides',
                                                       'Start_Station',
                                                       'End_Station'])

sorted_unique_routes_2018_df = pd.merge(sorted_unique_routes_2018_df, stations_sorted, left_on=["Start_Station"],
                          right_on=["ID"],
                          how = 'left')
            
sorted_unique_routes_2018_df = pd.merge(sorted_unique_routes_2018_df, stations_sorted, left_on=["End_Station"],
                          right_on=["ID"],
                          how = 'left')

sorted_unique_routes_2018_df.drop(['ID_x', 'ID_y'], inplace=True, axis=1)

sorted_unique_routes_2018_df = sorted_unique_routes_2018_df.rename(columns={"Name_x": "Start_Station_Name",
                                               'Lat_x' : 'Start_Station_Lat',
                                               'Lon_x' : 'Start_Station_Lon',
                                               'districtName_x' : 'Start_Colonia',
                                               "Name_y": "End_Station_Name",
                                               'Lat_y' : 'End_Station_Lat',
                                               'Lon_y' : 'End_Station_Lon',
                                               'districtName_y' : 'End_Colonia'})
sorted_unique_routes_2018_df.head()

,Total_Rides,Start_Station,End_Station,Start_Station_Name,Start_Station_Lat,Start_Station_Lon,Start_Colonia,End_Station_Name,End_Station_Lat,End_Station_Lon,End_Colonia
0,53,211,217,211 NEWTON-HORACIO,19.433607,-99.188528,Polanco,217 EULER-AV. HORACIO,19.432888,-99.183605,Polanco
1,51,1,18,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,Cuauhtemoc,18 REFORMA-RIO RHIN,19.428880,-99.164176,Cuauhtemoc
2,48,18,1,18 REFORMA-RIO RHIN,19.428880,-99.164176,Cuauhtemoc,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,Cuauhtemoc
3,47,107,113,107 TOLSA-BALDERAS,19.427329,-99.149971,Centro,113 GENERAL PRIM-VERSALLES,19.430369,-99.154419,Juarez
4,42,111,174,111 GUILLERMO PRIETO-JOAQUÍN VELÁZQUEZ DE LEÓN,19.438237,-99.165946,San Rafael,174 JOAQUÍN GARCIA-IGNACIO MANUEL ALTAMIRANO,19.441110,-99.161640,San Rafael


In [43]:
sorted_unique_routes_2018_df

,Total_Rides,Start_Station,End_Station,Start_Station_Name,Start_Station_Lat,Start_Station_Lon,Start_Colonia,End_Station_Name,End_Station_Lat,End_Station_Lon,End_Colonia
0,53,211,217,211 NEWTON-HORACIO,19.433607,-99.188528,Polanco,217 EULER-AV. HORACIO,19.432888,-99.183605,Polanco
1,51,1,18,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,Cuauhtemoc,18 REFORMA-RIO RHIN,19.428880,-99.164176,Cuauhtemoc
2,48,18,1,18 REFORMA-RIO RHIN,19.428880,-99.164176,Cuauhtemoc,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,Cuauhtemoc
3,47,107,113,107 TOLSA-BALDERAS,19.427329,-99.149971,Centro,113 GENERAL PRIM-VERSALLES,19.430369,-99.154419,Juarez
4,42,111,174,111 GUILLERMO PRIETO-JOAQUÍN VELÁZQUEZ DE LEÓN,19.438237,-99.165946,San Rafael,174 JOAQUÍN GARCIA-IGNACIO MANUEL ALTAMIRANO,19.441110,-99.161640,San Rafael
...,...,...,...,...,...,...,...,...,...,...,...
40694,1,311,288,311 CALLE 5-AVENIDA REVOLUCIÓN,19.393665,-99.185391,San Pedro de los Pinos,288 11 DE ABRIL-AVENIDA 2,19.396333,-99.183575,San Pedro de los Pinos
40695,1,201,475,201 LOPE DE VEGA-EJERCITO NACIONAL,19.437431,-99.187215,Polanco,475 LAGO MURITZ-LAGO ALBERTO,19.439249,-99.181065,Anahuac I
40696,1,461,194,461 PRESA DON MARTÍN-BOULEVARD MIGUEL DE CERVA...,19.440106,-99.210031,Irrigacion,194 CTO MAHATMA GANDHI-AV. PASEO DE LA REFORMA,19.424282,-99.180334,Bosque de Chapultepec
40697,1,355,373,355 CALIFORNIA-SACRAMENTO,19.382050,-99.173291,Insurgentes San Borja,373 PILARES-GABRIEL MANCERA,19.377968,-99.166663,Del Valle Centro


In [44]:
sorted_unique_routes_2018_df.to_json("Routes/sorted_unique_routes_2018.json", orient='index') 

In [45]:
routes_2020 = ecobici_rides_df_test.loc[ecobici_rides_df_test.Usage_Timestamp.dt.year==2020]

unique_routes_2020 = uniqueRoutes(routes_2020)
sorted_unique_routes_2020 = dict(sorted(unique_routes_2020.items(), key=lambda item: item[1], reverse=True))

lst = []

for (key, value) in sorted_unique_routes_2020.items():
   # Check if key is even then add pair to new dictionary
   lst.append([value, key[0], key[1]])

sorted_unique_routes_2020_df = pd.DataFrame(lst, columns=['Total_Rides',
                                                       'Start_Station',
                                                       'End_Station'])

sorted_unique_routes_2020_df = pd.merge(sorted_unique_routes_2020_df, stations_sorted, left_on=["Start_Station"],
                          right_on=["ID"],
                          how = 'left')
            
sorted_unique_routes_2020_df = pd.merge(sorted_unique_routes_2020_df, stations_sorted, left_on=["End_Station"],
                          right_on=["ID"],
                          how = 'left')

sorted_unique_routes_2020_df.drop(['ID_x', 'ID_y'], inplace=True, axis=1)

sorted_unique_routes_2020_df = sorted_unique_routes_2020_df.rename(columns={"Name_x": "Start_Station_Name",
                                               'Lat_x' : 'Start_Station_Lat',
                                               'Lon_x' : 'Start_Station_Lon',
                                               'districtName_x' : 'Start_Colonia',
                                               "Name_y": "End_Station_Name",
                                               'Lat_y' : 'End_Station_Lat',
                                               'Lon_y' : 'End_Station_Lon',
                                               'districtName_y' : 'End_Colonia'})

sorted_unique_routes_2020_df.to_json("Routes/sorted_unique_routes_2020.json", orient='index') 
sorted_unique_routes_2020_df.head()

,Total_Rides,Start_Station,End_Station,Start_Station_Name,Start_Station_Lat,Start_Station_Lon,Start_Colonia,End_Station_Name,End_Station_Lat,End_Station_Lon,End_Colonia
0,28,27,27,27 REFORMA-HAVRE,19.429115,-99.162614,Juarez,27 REFORMA-HAVRE,19.429115,-99.162614,Juarez
1,26,34,34,34 LIVERPOOL-GENOVA,19.424725,-99.163331,Juarez,34 LIVERPOOL-GENOVA,19.424725,-99.163331,Juarez
2,25,18,1,18 REFORMA-RIO RHIN,19.428880,-99.164176,Cuauhtemoc,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,Cuauhtemoc
3,24,33,33,E33 LONDRES-FLORENCIA,19.424234,-99.166431,Juarez,E33 LONDRES-FLORENCIA,19.424234,-99.166431,Juarez
4,23,139,139,E139 TONALÁ-GUANJUATO,19.416128,-99.162097,Roma Norte,E139 TONALÁ-GUANJUATO,19.416128,-99.162097,Roma Norte


In [46]:
routes_2012 = ecobici_rides_df_test.loc[ecobici_rides_df_test.Usage_Timestamp.dt.year==2012]

unique_routes_2012 = uniqueRoutes(routes_2012)
sorted_unique_routes_2012 = dict(sorted(unique_routes_2012.items(), key=lambda item: item[1], reverse=True))

lst = []

for (key, value) in sorted_unique_routes_2012.items():
   # Check if key is even then add pair to new dictionary
   lst.append([value, key[0], key[1]])

sorted_unique_routes_2012_df = pd.DataFrame(lst, columns=['Total_Rides',
                                                       'Start_Station',
                                                       'End_Station'])

sorted_unique_routes_2012_df = pd.merge(sorted_unique_routes_2012_df, stations_sorted, left_on=["Start_Station"],
                          right_on=["ID"],
                          how = 'left')
            
sorted_unique_routes_2012_df = pd.merge(sorted_unique_routes_2012_df, stations_sorted, left_on=["End_Station"],
                          right_on=["ID"],
                          how = 'left')

sorted_unique_routes_2012_df.drop(['ID_x', 'ID_y'], inplace=True, axis=1)

sorted_unique_routes_2012_df = sorted_unique_routes_2012_df.rename(columns={"Name_x": "Start_Station_Name",
                                               'Lat_x' : 'Start_Station_Lat',
                                               'Lon_x' : 'Start_Station_Lon',
                                               'districtName_x' : 'Start_Colonia',
                                               "Name_y": "End_Station_Name",
                                               'Lat_y' : 'End_Station_Lat',
                                               'Lon_y' : 'End_Station_Lon',
                                               'districtName_y' : 'End_Colonia'})

sorted_unique_routes_2012_df.to_json("Routes/sorted_unique_routes_2012.json", orient='index') 
sorted_unique_routes_2012_df.head()

,Total_Rides,Start_Station,End_Station,Start_Station_Name,Start_Station_Lat,Start_Station_Lon,Start_Colonia,End_Station_Name,End_Station_Lat,End_Station_Lon,End_Colonia
0,65,1,18,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,Cuauhtemoc,18 REFORMA-RIO RHIN,19.428880,-99.164176,Cuauhtemoc
1,43,18,1,18 REFORMA-RIO RHIN,19.428880,-99.164176,Cuauhtemoc,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,Cuauhtemoc
2,41,1,27,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,Cuauhtemoc,27 REFORMA-HAVRE,19.429115,-99.162614,Juarez
3,41,85,85,85 ROSAS MORENO-SULLIVAN,19.434250,-99.162508,San Rafael,85 ROSAS MORENO-SULLIVAN,19.434250,-99.162508,San Rafael
4,41,41,3,41 REFORMA-AV. DE LA REPUBLICA,19.435637,-99.150137,Tabacalera,3 REFORMA-INSURGENTES,19.431655,-99.158668,Juarez


In [47]:
routes_2014 = ecobici_rides_df_test.loc[ecobici_rides_df_test.Usage_Timestamp.dt.year==2014]

unique_routes_2014 = uniqueRoutes(routes_2014)
sorted_unique_routes_2014 = dict(sorted(unique_routes_2014.items(), key=lambda item: item[1], reverse=True))

lst = []

for (key, value) in sorted_unique_routes_2014.items():
   # Check if key is even then add pair to new dictionary
   lst.append([value, key[0], key[1]])

sorted_unique_routes_2014_df = pd.DataFrame(lst, columns=['Total_Rides',
                                                       'Start_Station',
                                                       'End_Station'])

sorted_unique_routes_2014_df = pd.merge(sorted_unique_routes_2014_df, stations_sorted, left_on=["Start_Station"],
                          right_on=["ID"],
                          how = 'left')
            
sorted_unique_routes_2014_df = pd.merge(sorted_unique_routes_2014_df, stations_sorted, left_on=["End_Station"],
                          right_on=["ID"],
                          how = 'left')

sorted_unique_routes_2014_df.drop(['ID_x', 'ID_y'], inplace=True, axis=1)

sorted_unique_routes_2014_df = sorted_unique_routes_2014_df.rename(columns={"Name_x": "Start_Station_Name",
                                               'Lat_x' : 'Start_Station_Lat',
                                               'Lon_x' : 'Start_Station_Lon',
                                               'districtName_x' : 'Start_Colonia',
                                               "Name_y": "End_Station_Name",
                                               'Lat_y' : 'End_Station_Lat',
                                               'Lon_y' : 'End_Station_Lon',
                                               'districtName_y' : 'End_Colonia'})

sorted_unique_routes_2014_df.to_json("Routes/sorted_unique_routes_2014.json", orient='index') 
sorted_unique_routes_2014_df.head()

,Total_Rides,Start_Station,End_Station,Start_Station_Name,Start_Station_Lat,Start_Station_Lon,Start_Colonia,End_Station_Name,End_Station_Lat,End_Station_Lon,End_Colonia
0,63,41,3,41 REFORMA-AV. DE LA REPUBLICA,19.435637,-99.150137,Tabacalera,3 REFORMA-INSURGENTES,19.431655,-99.158668,Juarez
1,63,183,174,183 GABINO BARRERA-GUILLERMO PRIETO,19.437670,-99.164020,San Rafael,174 JOAQUÍN GARCIA-IGNACIO MANUEL ALTAMIRANO,19.441110,-99.161640,San Rafael
2,60,211,217,211 NEWTON-HORACIO,19.433607,-99.188528,Polanco,217 EULER-AV. HORACIO,19.432888,-99.183605,Polanco
3,53,21,27,21 REFORMA-DUBLIN,19.424720,-99.172060,Juarez,27 REFORMA-HAVRE,19.429115,-99.162614,Juarez
4,49,36,53,36 PUEBLA-VERACRUZ,19.419725,-99.175778,Piedad Narvarte,53 FERNANDO MONTES DE OCA-TULA,19.414773,-99.178433,Condesa


In [48]:
routes_2016 = ecobici_rides_df_test.loc[ecobici_rides_df_test.Usage_Timestamp.dt.year==2016]

unique_routes_2016 = uniqueRoutes(routes_2016)
sorted_unique_routes_2016 = dict(sorted(unique_routes_2016.items(), key=lambda item: item[1], reverse=True))

lst = []

for (key, value) in sorted_unique_routes_2016.items():
   # Check if key is even then add pair to new dictionary
   lst.append([value, key[0], key[1]])

sorted_unique_routes_2016_df = pd.DataFrame(lst, columns=['Total_Rides',
                                                       'Start_Station',
                                                       'End_Station'])

sorted_unique_routes_2016_df = pd.merge(sorted_unique_routes_2016_df, stations_sorted, left_on=["Start_Station"],
                          right_on=["ID"],
                          how = 'left')
            
sorted_unique_routes_2016_df = pd.merge(sorted_unique_routes_2016_df, stations_sorted, left_on=["End_Station"],
                          right_on=["ID"],
                          how = 'left')

sorted_unique_routes_2016_df.drop(['ID_x', 'ID_y'], inplace=True, axis=1)

sorted_unique_routes_2016_df = sorted_unique_routes_2016_df.rename(columns={"Name_x": "Start_Station_Name",
                                               'Lat_x' : 'Start_Station_Lat',
                                               'Lon_x' : 'Start_Station_Lon',
                                               'districtName_x' : 'Start_Colonia',
                                               "Name_y": "End_Station_Name",
                                               'Lat_y' : 'End_Station_Lat',
                                               'Lon_y' : 'End_Station_Lon',
                                               'districtName_y' : 'End_Colonia'})

sorted_unique_routes_2016_df.to_json("Routes/sorted_unique_routes_2016.json", orient='index') 
sorted_unique_routes_2016_df.head()

,Total_Rides,Start_Station,End_Station,Start_Station_Name,Start_Station_Lat,Start_Station_Lon,Start_Colonia,End_Station_Name,End_Station_Lat,End_Station_Lon,End_Colonia
0,72,211,217,211 NEWTON-HORACIO,19.433607,-99.188528,Polanco,217 EULER-AV. HORACIO,19.432888,-99.183605,Polanco
1,58,174,183,174 JOAQUÍN GARCIA-IGNACIO MANUEL ALTAMIRANO,19.441110,-99.161640,San Rafael,183 GABINO BARRERA-GUILLERMO PRIETO,19.437670,-99.164020,San Rafael
2,51,174,257,174 JOAQUÍN GARCIA-IGNACIO MANUEL ALTAMIRANO,19.441110,-99.161640,San Rafael,E257 MANUEL MARIA CONTRERAS-VILLALONGIN,19.433800,-99.166220,Cuauhtemoc
3,48,183,174,183 GABINO BARRERA-GUILLERMO PRIETO,19.437670,-99.164020,San Rafael,174 JOAQUÍN GARCIA-IGNACIO MANUEL ALTAMIRANO,19.441110,-99.161640,San Rafael
4,47,217,211,217 EULER-AV. HORACIO,19.432888,-99.183605,Polanco,211 NEWTON-HORACIO,19.433607,-99.188528,Polanco
